In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import random_correlation
from tqdm import tqdm

from curves import *
from volatility import *
from correlation import *
from ratemodels import *

In [3]:
libor_curve, ois_curve = curve_builder()
cap_vol = pd.read_excel('project_data.xlsx',sheet_name='caps vol', index_col='Tenor')
t, vol = cap_vol.index.values, cap_vol.ATM.values/100
historical_rates = pd.read_excel('project_data.xlsx',sheet_name='historical f', index_col='Date')
spot_data = {'ois_curve': ois_curve,
            'libor_curve':libor_curve,
            'cap_vol': (t, vol),
            'historical_rates': historical_rates}
options = {'r_interp':  {'kernel':'MCS','extrapolate':True},
           'df_interp': {'kernel':'MCS','extrapolate':True},
           'vol_interp':{'kernel':'MCS','extrapolate':True},
           'corr_factors': 'all'}
vol_boost =  CapletVolStrip(spot_data, options['vol_interp'])
t, stripped_vol = vol_boost.strip(spot_data['cap_vol'][0],spot_data['cap_vol'][1], display=False)
vol_func = ParametricVolatility()
vol_func.fit(t,stripped_vol)

array([0.        , 0.3171615 , 0.56337124, 0.2248542 ])

In [4]:
x = 5
options['T'] = x
model = LiborMarketModel(options)
model.calibrate(spot_data)
model.simulate(n_sim=100000)

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [02:49<00:00, 588.97it/s]


In [5]:
print(pd.DataFrame(model.f[:,:,0]))
print(model.f[-2,-1,0])

           0         1         2         3         4         5         6  \
0   0.019112  0.017221  0.016944  0.016606  0.017083  0.017182  0.017693   
1        NaN  0.016606  0.016806  0.014392  0.014271  0.014336  0.015778   
2        NaN       NaN  0.016761  0.015496  0.012615  0.012303  0.015105   
3        NaN       NaN       NaN  0.019851  0.017083  0.017732  0.017402   
4        NaN       NaN       NaN       NaN  0.012517  0.011756  0.011680   
5        NaN       NaN       NaN       NaN       NaN  0.008342  0.007829   
6        NaN       NaN       NaN       NaN       NaN       NaN  0.010035   
7        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
8        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
10       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

           7         8         9  
0   0.018284  0.018504  0.018888  
1   0.016290  0.0

In [7]:
amount = 1000000
lmm_price = []
bs_price = []
bs_vol = []
lmm_vol = []
f = libor_curve.forward_rate(x-0.5,x)
df = ois_curve.discount(x)
tmp = model.f[-2,-1,:]-f
tmp[tmp<0] = 0
L = np.mean(tmp)
lmm_price.append(L*0.5*amount*df)

#vol = vol_func.caplet_vol(0,x-0.5,vol_func.params)
vol = vol_func.caplet_vol(0,x-0.5)
bs_price.append(vol_boost.black_price(0.5,x,f,f,vol)*0.5*amount*df)
lmm_price = np.array(lmm_price)
bs_price = np.array(bs_price)
print(pd.DataFrame({'LMM':lmm_price,'BS':bs_price,'%diff':(lmm_price/bs_price-1)*100}))

           LMM           BS     %diff
0  2822.788794  2818.200709  0.162802


In [57]:
print(f)
print(np.mean(model.f[-2,-1,:]))

0.018887645295422417
0.018872094389659883


In [27]:
def caplet_vol(t, T):
        T = np.arange(0.5,T+0.5,0.5)
        a = vol_func.instant_vol(t,T,vol_func.params)
        q = [0.5*v**2 for v in a]
        for i, x in enumerate(T):
            tmp = np.sqrt(np.sum(q[:i+1])/x)
        return tmp

In [28]:
caplet_vol(0,x-0.5)

0.3927406587821774

In [9]:
vol_func.instant_vol(0,5,vol_func.params)

0.3182734489592359

In [ ]:
vol_func.caplet_vol(0,1,vol_func.params)

In [ ]:
t = np.arange(0.5,2,0.5)
np.mean(vol_func(0.5,t)*np.sqrt(0.5))/np.sqrt(0.5)

In [ ]:
model.f[:,:,0]

In [ ]:
np.sqrt((0.5)*(vol_func(0,0.5)**2+vol_func(0,1)**2))


In [ ]:
vol_func.caplet_vol(0,1,vol_func.params)

In [ ]:
vol_func(0,0.5)